## Noise Reduction

#### Packages

In [ ]:
import IPython
from pydub import AudioSegment
import numpy as np
from scipy.io import wavfile
from plotly.offline import init_notebook_mode
import scipy.signal
import matplotlib.pyplot as plt
import librosa
import plotly.graph_objs as go
import plotly
import os
import time
from datetime import timedelta as td

In [ ]:
import numpy as np
import scipy.signal
import librosa
import matplotlib.pyplot as plt

# === Helper Functions ===
def _stft(y, n_fft, hop_length, win_length):
    return librosa.stft(y=y, n_fft=n_fft, hop_length=hop_length, win_length=win_length)

def _istft(y, hop_length, win_length):
    return librosa.istft(y, hop_length=hop_length, win_length=win_length)

def _amp_to_db(x):
    return librosa.core.amplitude_to_db(x, ref=1.0, amin=1e-20, top_db=80.0)

def _db_to_amp(x):
    return librosa.core.db_to_amplitude(x, ref=1.0)

def plot_spectrogram(signal, title):
    fig, ax = plt.subplots(figsize=(20, 4))
    cax = ax.matshow(
        signal,
        origin="lower",
        aspect="auto",
        cmap=plt.cm.seismic,
        vmin=-1 * np.max(np.abs(signal)),
        vmax=np.max(np.abs(signal)),
    )
    fig.colorbar(cax)
    ax.set_title(title)
    plt.tight_layout()
    plt.show()

def plot_statistics_and_filter(mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter):
    fig, ax = plt.subplots(ncols=2, figsize=(20, 4))
    ax[0].plot(mean_freq_noise, label="Mean power of noise")
    ax[0].plot(std_freq_noise, label="Std. power of noise")
    ax[0].plot(noise_thresh, label="Noise threshold (by frequency)")
    ax[0].set_title("Threshold for mask")
    ax[0].legend()
    cax = ax[1].matshow(smoothing_filter, origin="lower")
    fig.colorbar(cax)
    ax[1].set_title("Filter for smoothing mask")
    plt.show()

# === Main Denoising Function ===
def removeNoise(
    audio_clip,
    noise_clip,
    n_grad_freq=2,
    n_grad_time=4,
    n_fft=2048,
    win_length=2048,
    hop_length=512,
    n_std_thresh=1.0,
    prop_decrease=0.8,
    verbose=False,
    visual=False,
):
    # STFT sur le bruit
    noise_stft = _stft(noise_clip, n_fft, hop_length, win_length)
    noise_stft_db = _amp_to_db(np.abs(noise_stft))
    mean_freq_noise = np.mean(noise_stft_db, axis=1)
    std_freq_noise = np.std(noise_stft_db, axis=1)
    noise_thresh = mean_freq_noise + std_freq_noise * n_std_thresh

    # STFT sur le signal
    sig_stft = _stft(audio_clip, n_fft, hop_length, win_length)
    sig_stft_db = _amp_to_db(np.abs(sig_stft))

    # Création du filtre de lissage
    smoothing_filter = np.outer(
        np.concatenate([
            np.linspace(0, 1, n_grad_freq + 1, endpoint=False),
            np.linspace(1, 0, n_grad_freq + 2)
        ])[1:-1],
        np.concatenate([
            np.linspace(0, 1, n_grad_time + 1, endpoint=False),
            np.linspace(1, 0, n_grad_time + 2)
        ])[1:-1],
    )
    smoothing_filter /= np.sum(smoothing_filter)

    # Création du masque
    db_thresh = np.repeat(noise_thresh.reshape([len(mean_freq_noise), 1]), sig_stft_db.shape[1], axis=1)
    sig_mask = sig_stft_db < db_thresh
    sig_mask = scipy.signal.fftconvolve(sig_mask, smoothing_filter, mode="same")
    sig_mask *= prop_decrease

    # Application douce du masque avec la phase d'origine
    sig_stft_db_masked = sig_stft_db * (1 - sig_mask * prop_decrease)
    sig_phase = np.angle(sig_stft)
    sig_stft_amp = _db_to_amp(sig_stft_db_masked) * np.exp(1j * sig_phase)

    # Reconstruction du signal
    recovered_signal = _istft(sig_stft_amp, hop_length, win_length)

    if visual:
        plot_spectrogram(noise_stft_db, title="Noise Spectrogram")
        plot_statistics_and_filter(mean_freq_noise, std_freq_noise, noise_thresh, smoothing_filter)
        plot_spectrogram(sig_stft_db, title="Original Signal Spectrogram")
        plot_spectrogram(sig_stft_db_masked, title="Denoised Signal Spectrogram")

    return recovered_signal


In [ ]:
import librosa
import soundfile as sf

# Charger les fichiers audio
audio_path = '/kaggle/input/daic-woz/354_P/354_AUDIO.wav'
noise_path = '/kaggle/input/white-noise/TEST_Bruit blanc (ID 1037)_LS.wav'

# Charger les signaux avec la même fréquence d’échantillonnage
audio_clip, sr = librosa.load(audio_path, sr=None)
noise_clip, _ = librosa.load(noise_path, sr=sr)

# Appliquer la réduction de bruit
denoised_audio = removeNoise(
    audio_clip=audio_clip,
    noise_clip=noise_clip,
    n_std_thresh=1.5,      # plus agressif si nécessaire
    prop_decrease=0.8,     # proportion du bruit à retirer
    visual=True            # pour afficher les spectrogrammes
)

# Sauvegarder le signal débruité
sf.write('denoised_output.wav', denoised_audio, sr)
print("✅ Fichier nettoyé enregistré sous : denoised_output.wav")


In [ ]:
from IPython.display import Audio

# Lire le fichier audio nettoyé
Audio("denoised_output.wav")

In [ ]:
Audio("/kaggle/input/daic-woz/354_P/354_AUDIO.wav")